# Homework 06 — Trees and Ensembles (Solutions)

This notebook implements the solution to `cohorts/2025/06-trees/homework.md` using the Car Fuel Efficiency dataset.

Target: predict `fuel_efficiency_mpg`.

General steps:
- Load data and fill missing values with 0.
- Split 60%/20%/20% with `random_state=1`.
- Vectorize with `DictVectorizer(sparse=True)`.
- Answer Q1–Q6 with reproducible code.

In [2]:
# Imports
import os
import math
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

RANDOM_STATE = 1
np.random.seed(RANDOM_STATE)

## Data loading and preparation
- Read the CSV locally if present, otherwise load from the URL.
- Fill missing values with 0 (as per assignment).
- Identify the target column (ideally `fuel_efficiency_mpg`).

In [3]:
DATA_URL = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv'
DATA_PATH = 'car_fuel_efficiency.csv'

# Load dataset (local or remote)
if os.path.exists(DATA_PATH):
    df = pd.read_csv(DATA_PATH)
else:
    # If you prefer, download first with `wget` as in the assignment.
    df = pd.read_csv(DATA_URL)

# Fill missing values with 0
df = df.copy()
df.fillna(0, inplace=True)

# Identify preferred target column
target_candidates = ["fuel_efficiency_mpg", "combined_mpg", "fuel_efficiency"]
target = None
for c in target_candidates:
    if c in df.columns:
        target = c
        break
if target is None:
    # Fallback: search for a column with 'mpg' or 'efficien' in the name
    for c in df.columns:
        cl = c.lower()
        if 'mpg' in cl or 'efficien' in cl:
            target = c
            break

assert target is not None, 'Could not find the target column (MPG/efficiency).'
target


'fuel_efficiency_mpg'

In [4]:
# Split 60/20/20 with random_state=1
df_full = df.copy()
y = df_full[target].values
X = df_full.drop(columns=[target])

# First: hold out 20% for test
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)
# Then: from the remaining 80%, hold out 25% for validation -> 0.8 * 0.25 = 0.2
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=RANDOM_STATE)

len(X_train), len(X_val), len(X_test)


(5822, 1941, 1941)

In [5]:
# Vectorization with DictVectorizer(sparse=True)
dv = DictVectorizer(sparse=True)

train_dicts = X_train.to_dict(orient='records')
val_dicts   = X_val.to_dict(orient='records')
test_dicts  = X_test.to_dict(orient='records')

X_train_dv = dv.fit_transform(train_dicts)
X_val_dv   = dv.transform(val_dicts)
X_test_dv  = dv.transform(test_dicts)

X_train_dv.shape, X_val_dv.shape, X_test_dv.shape


((5822, 14), (1941, 14), (1941, 14))

In [6]:
def rmse(y_true, y_pred):
    return math.sqrt(mean_squared_error(y_true, y_pred))

def base_name(feat_name):
    # Aggregate DV feature names (e.g., 'origin=Asia' -> 'origin')
    return feat_name.split('=')[0] if isinstance(feat_name, str) else feat_name

def pick_first_available(cols, candidates):
    for c in candidates:
        if c in cols:
            return c
    return None


## Q1 — Decision tree (max_depth=1)
Train a `DecisionTreeRegressor(max_depth=1)` and identify the feature used at the root split.

Because vectorization expands categories (e.g., `origin=Asia`), we map the root index back to the original feature name.

In [7]:
dt = DecisionTreeRegressor(max_depth=1, random_state=RANDOM_STATE)
dt.fit(X_train_dv, y_train)

root_idx = dt.tree_.feature[0]  # index of the DV feature
root_feat = dv.feature_names_[root_idx]
root_base = base_name(root_feat)
root_feat, root_base


('vehicle_weight', 'vehicle_weight')

Answer (Q1): the feature used at the root is printed in `root_base`.
Select the corresponding option (`vehicle_weight`, `model_year`, `origin`, or `fuel_type`).

## Q2 — Random Forest (n_estimators=10)
Train `RandomForestRegressor(n_estimators=10, random_state=1, n_jobs=-1)` and compute RMSE on the validation set.

In [8]:
rf10 = RandomForestRegressor(n_estimators=10, random_state=RANDOM_STATE, n_jobs=-1)
rf10.fit(X_train_dv, y_train)
val_pred = rf10.predict(X_val_dv)
rmse_q2 = rmse(y_val, val_pred)
rmse_q2


0.4586615458484907

Answer (Q2): choose among 0.045, 0.45, 4.5 or 45.0 the closest to `rmse_q2`.
(Typically, realistic values here are around 4–5).

## Q3 — Effect of n_estimators (10 → 200)
Vary `n_estimators` from 10 to 200 (step 10), `random_state=1`, evaluate on validation and find after which value RMSE stops improving (3 decimal places).

In [9]:
vals = list(range(10, 201, 10))
rmse_by_n = []

for n in vals:
    model = RandomForestRegressor(n_estimators=n, random_state=RANDOM_STATE, n_jobs=-1)
    model.fit(X_train_dv, y_train)
    pred = model.predict(X_val_dv)
    r = rmse(y_val, pred)
    rmse_by_n.append(r)

list(zip(vals, rmse_by_n))[:5], len(rmse_by_n)


([(10, 0.4586615458484907),
  (20, 0.4536799102144079),
  (30, 0.4511716029987014),
  (40, 0.4483573590280684),
  (50, 0.446179229382576)],
 20)

In [10]:
# Find the last point where RMSE (rounded to 3 decimals) improves
best_rmse_round3 = float('inf')
last_improving_n = vals[0]
rmse_round3 = [round(x, 3) for x in rmse_by_n]

for n, r in zip(vals, rmse_round3):
    if r < best_rmse_round3:
        best_rmse_round3 = r
        last_improving_n = n

best_rmse_round3, last_improving_n


(0.443, 150)

Answer (Q3): `last_improving_n` (if it never stops improving up to 200, answer 200).

## Q4 — Best max_depth by mean RMSE
For each `max_depth` in `[10, 15, 20, 25]`, compute the mean RMSE on validation by sweeping `n_estimators` from 10 to 200 (step 10). Choose the `max_depth` with the lowest mean.

In [11]:
depths = [10, 15, 20, 25]
mean_rmse_by_depth = {}

for d in depths:
    rmses = []
    for n in vals:
        model = RandomForestRegressor(n_estimators=n, max_depth=d, random_state=RANDOM_STATE, n_jobs=-1)
        model.fit(X_train_dv, y_train)
        pred = model.predict(X_val_dv)
        rmses.append(rmse(y_val, pred))
    mean_rmse_by_depth[d] = float(np.mean(rmses))

mean_rmse_by_depth


{10: 0.44187929925252156,
 15: 0.44561628816456206,
 20: 0.4456793443309614,
 25: 0.44570249863475137}

In [12]:
best_depth = min(mean_rmse_by_depth, key=mean_rmse_by_depth.get)
best_depth, mean_rmse_by_depth[best_depth]


(10, 0.44187929925252156)

Answer (Q4): `best_depth` among 10, 15, 20, 25.

## Q5 — Feature importances (Random Forest)
Train `RandomForestRegressor(n_estimators=10, max_depth=20, random_state=1, n_jobs=-1)` and inspect `feature_importances_`.

Since `DictVectorizer` creates multiple columns for categories, aggregate importance by base feature name (before `=`) and compare among:
`vehicle_weight`, `horsepower`, `acceleration`, and `engine_displacement`.

In [13]:
rf_imp = RandomForestRegressor(n_estimators=10, max_depth=20, random_state=RANDOM_STATE, n_jobs=-1)
rf_imp.fit(X_train_dv, y_train)

feat_names = dv.feature_names_
importances = rf_imp.feature_importances_

# Aggregate by base name
agg = {}
for fn, imp in zip(feat_names, importances):
    b = base_name(fn)
    agg[b] = agg.get(b, 0.0) + float(imp)

# Candidates (with possible synonyms)
cands = {
    'vehicle_weight': ['vehicle_weight', 'weight', 'curb_weight'],
    'horsepower': ['horsepower', 'engine_hp', 'engine_power'],
    'acceleration': ['acceleration'],
    'engine_displacement': ['engine_displacement', 'displacement']
}

cand_scores = {}
for key, aliases in cands.items():
    score = 0.0
    for a in aliases:
        score += agg.get(a, 0.0)
    cand_scores[key] = score

cand_scores


{'vehicle_weight': 0.9591531737242687,
 'horsepower': 0.01606583100118693,
 'acceleration': 0.011489660517019416,
 'engine_displacement': 0.0032794702827490425}

In [14]:
most_important = max(cand_scores, key=cand_scores.get)
most_important, cand_scores[most_important]


('vehicle_weight', 0.9591531737242687)

Answer (Q5): `most_important` among the 4 options provided.

## Q6 — XGBoost: compare eta=0.3 vs 0.1
Train for 100 rounds with the given parameters, create `DMatrix` for train and validation, a `watchlist`, and compare validation RMSE for `eta=0.3` and `eta=0.1`.

In [15]:
# Install xgboost if needed (uncomment on your machine)
# %pip install xgboost

try:
    import xgboost as xgb
except Exception as e:
    raise RuntimeError("xgboost is not installed. Install with `pip install xgboost`.")

dtrain = xgb.DMatrix(X_train_dv, label=y_train)
dval   = xgb.DMatrix(X_val_dv,   label=y_val)
watchlist = [(dtrain, 'train'), (dval, 'val')]

base_params = {
    'max_depth': 6,
    'min_child_weight': 1,
    'objective': 'reg:squarederror',
    'nthread': 8,
    'seed': RANDOM_STATE,
    'verbosity': 1,
}

def train_and_eval(eta):
    params = dict(base_params)
    params['eta'] = eta
    booster = xgb.train(params, dtrain, num_boost_round=100, evals=watchlist, verbose_eval=False)
    pred = booster.predict(dval)
    return rmse(y_val, pred)

rmse_eta_03 = train_and_eval(0.3)
rmse_eta_01 = train_and_eval(0.1)
rmse_eta_03, rmse_eta_01


RuntimeError: xgboost is not installed. Install with `pip install xgboost`.

Answer (Q6): choose `0.3`, `0.1` or `Both` depending on which RMSE (`rmse_eta_03` vs `rmse_eta_01`) is smaller (or if they are equal).

## Answer Summary
This cell selects the final options based on the computed results above. If Q6 fails due to missing OpenMP on macOS, install it (see the note below) and re-run the XGBoost cell.

In [16]:
# Q1: feature used at the root
q1_answer = root_base  # one of: 'vehicle_weight', 'model_year', 'origin', 'fuel_type'

# Q2: RF(10) RMSE -> choose closest among [0.045, 0.45, 4.5, 45.0]
q2_options = [0.045, 0.45, 4.5, 45.0]
q2_choice = min(q2_options, key=lambda x: abs(x - rmse_q2))

# Q3: last improving n_estimators -> choose closest among [10, 25, 80, 200]
q3_options = [10, 25, 80, 200]
q3_choice = min(q3_options, key=lambda x: abs(x - last_improving_n))

# Q4: best max_depth among [10, 15, 20, 25]
q4_choice = best_depth

# Q5: most important feature among the four listed
q5_answer = most_important

# Q6: try to compute; if not available, leave None
q6_answer = None
try:
    _ = rmse_eta_03; _ = rmse_eta_01
    q6_answer = '0.3' if rmse_eta_03 < rmse_eta_01 else ('0.1' if rmse_eta_01 < rmse_eta_03 else 'Both')
except Exception:
    try:
        import xgboost as xgb  # noqa: F401
    except Exception:
        q6_answer = None

answers = {
    'Q1_feature': q1_answer,
    'Q2_RMSE': float(rmse_q2),
    'Q2_choice': q2_choice,
    'Q3_last_improving_n': int(last_improving_n),
    'Q3_choice': int(q3_choice),
    'Q4_best_max_depth': int(q4_choice),
    'Q5_most_important': q5_answer,
    'Q6_best_eta': q6_answer,
}
answers


{'Q1_feature': 'vehicle_weight',
 'Q2_RMSE': 0.4586615458484907,
 'Q2_choice': 0.45,
 'Q3_last_improving_n': 150,
 'Q3_choice': 200,
 'Q4_best_max_depth': 10,
 'Q5_most_important': 'vehicle_weight',
 'Q6_best_eta': None}

### Note on XGBoost on macOS
If you see an error about `libomp.dylib`, install the OpenMP runtime and re-run Q6:
- Homebrew: `brew install libomp`
- Or use a Conda environment where `xgboost` comes with OpenMP support

After installation, restart the kernel and execute the Q6 cells.